In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/10-monkey-species/monkey_labels.txt
/kaggle/input/10-monkey-species/training/training/n5/n5027.jpg
/kaggle/input/10-monkey-species/training/training/n5/n5161.jpg
/kaggle/input/10-monkey-species/training/training/n5/n5021.jpg
/kaggle/input/10-monkey-species/training/training/n5/n5154.jpg
/kaggle/input/10-monkey-species/training/training/n5/n5071.jpg
/kaggle/input/10-monkey-species/training/training/n5/n5145.jpg
/kaggle/input/10-monkey-species/training/training/n5/n5038.jpg
/kaggle/input/10-monkey-species/training/training/n5/n5020.jpg
/kaggle/input/10-monkey-species/training/training/n5/n5054.jpg
/kaggle/input/10-monkey-species/training/training/n5/n5019.jpg
/kaggle/input/10-monkey-species/training/training/n5/n5044.jpg
/kaggle/input/10-monkey-species/training/training/n5/n5159.jpg
/kaggle/input/10-monkey-species/training/training/n5/n5018.jpg
/kaggle/input/10-monkey-species/training/training/n5/n5110.jpg
/kaggle/input/10-monkey-species/training/training/n5/n5047.jpg
/kagg

In [2]:
from keras.applications import MobileNet
img_rows=224
img_cols=224
MobileNet=MobileNet(include_top=False, weights='imagenet',input_shape=(img_rows,img_cols,3))
for layer in MobileNet.layers:
    layer.trainable=False
for (i,layer) in enumerate(MobileNet.layers):
    print(str(i)+" "+layer.__class__.__name__,layer.trainable)

Using TensorFlow backend.


17227776/17225924 [==============================] - 1s 0us/step
0 InputLayer False
1 ZeroPadding2D False
2 Conv2D False
3 BatchNormalization False
4 ReLU False
5 DepthwiseConv2D False
6 BatchNormalization False
7 ReLU False
8 Conv2D False
9 BatchNormalization False
10 ReLU False
11 ZeroPadding2D False
12 DepthwiseConv2D False
13 BatchNormalization False
14 ReLU False
15 Conv2D False
16 BatchNormalization False
17 ReLU False
18 DepthwiseConv2D False
19 BatchNormalization False
20 ReLU False
21 Conv2D False
22 BatchNormalization False
23 ReLU False
24 ZeroPadding2D False
25 DepthwiseConv2D False
26 BatchNormalization False
27 ReLU False
28 Conv2D False
29 BatchNormalization False
30 ReLU False
31 DepthwiseConv2D False
32 BatchNormalization False
33 ReLU False
34 Conv2D False
35 BatchNormalization False
36 ReLU False
37 ZeroPadding2D False
38 DepthwiseConv2D False
39 BatchNormalization False
40 ReLU False
41 Conv2D False
42 BatchNormalization False
43 ReLU False
44 DepthwiseConv2D False


In [3]:
def addModel(my_model,num_classes):
    top_model = my_model.output
    top_model = GlobalAveragePooling2D()(top_model)
    top_model = Dense(1024,activation='relu')(top_model)
    top_model = Dense(1024,activation='relu')(top_model)
    top_model = Dense(512,activation='relu')(top_model)
    top_model = Dense(num_classes,activation='softmax')(top_model)
    return top_model

In [4]:
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Conv2D,GlobalAveragePooling2D,Dense,Flatten,Dropout,Activation
from keras.models import Sequential, Model
from keras.optimizers import SGD, RMSprop, Adam
from keras import backend as K

In [5]:
num_classes=10
layers_bottom=addModel(MobileNet,num_classes)
model=Model(inputs=MobileNet.input, outputs=layers_bottom)


In [6]:
train_data='../input/10-monkey-species/training/training'
test_data='../input/10-monkey-species/validation/validation'

In [7]:
train_datagen=ImageDataGenerator(rescale=1./255,width_shift_range=0.3,height_shift_range=0.3,
                                horizontal_flip=True,rotation_range=45, fill_mode='nearest')

In [8]:
train=train_datagen.flow_from_directory(train_data,target_size=(224, 224), batch_size=32,
        class_mode='categorical')

Found 1098 images belonging to 10 classes.


In [9]:
test_datagen=ImageDataGenerator(rescale=1./255)

In [10]:
test=test_datagen.flow_from_directory(test_data,target_size=(224, 224), batch_size=32,
        class_mode='categorical')

Found 272 images belonging to 10 classes.


In [11]:
from keras.optimizers import RMSprop
from keras.callbacks import ModelCheckpoint, EarlyStopping
earlystopping=EarlyStopping(monitor='val_loss',min_delta=0,patience=3,verbose=1,restore_best_weights=True)

In [12]:
model.compile(loss='categorical_crossentropy',optimizer=RMSprop(lr=0.001),metrics=['accuracy'])

In [13]:
callbacks=[earlystopping]
n_samples=1098
nv_samples=272
batch_size=16
epochs=5
model.fit_generator(train,
        steps_per_epoch=n_samples // batch_size,
        epochs=epochs,
        callbacks=callbacks,
        validation_data=test,
        validation_steps=nv_samples // batch_size)


Epoch 1/5
68/68 [==============================] - 96s 1s/step - loss: 1.9508 - accuracy: 0.4611 - val_loss: 0.2319 - val_accuracy: 0.8220
Epoch 2/5
68/68 [==============================] - 90s 1s/step - loss: 0.7450 - accuracy: 0.7795 - val_loss: 0.1060 - val_accuracy: 0.9570
Epoch 3/5
68/68 [==============================] - 89s 1s/step - loss: 0.5354 - accuracy: 0.8499 - val_loss: 0.7140 - val_accuracy: 0.9434
Epoch 4/5
68/68 [==============================] - 87s 1s/step - loss: 0.4356 - accuracy: 0.8705 - val_loss: 0.5316 - val_accuracy: 0.9238
Epoch 5/5
68/68 [==============================] - 86s 1s/step - loss: 0.3861 - accuracy: 0.8918 - val_loss: 0.8826 - val_accuracy: 0.8438
Restoring model weights from the end of the best epoch
Epoch 00005: early stopping


In [14]:

Y_pred = model.predict_generator(test, nv_samples // batch_size+1)
y_pred = np.argmax(Y_pred, axis=1)


In [15]:
y_pred

array([8, 9, 9, 6, 9, 0, 3, 9, 7, 3, 4, 2, 5, 7, 7, 4, 2, 2, 6, 1, 9, 3,
       3, 4, 7, 4, 6, 4, 7, 5, 6, 9, 1, 2, 9, 3, 1, 6, 8, 4, 5, 2, 5, 5,
       9, 2, 8, 9, 2, 8, 0, 5, 9, 9, 6, 3, 7, 9, 3, 1, 5, 4, 6, 3, 9, 6,
       6, 5, 2, 0, 7, 3, 0, 1, 8, 0, 8, 0, 3, 0, 7, 5, 2, 1, 2, 8, 0, 2,
       4, 3, 6, 7, 6, 3, 7, 8, 4, 2, 4, 2, 2, 2, 6, 1, 1, 3, 6, 7, 5, 0,
       4, 0, 9, 9, 9, 7, 7, 8, 8, 1, 9, 1, 8, 7, 2, 0, 7, 5, 6, 2, 3, 4,
       2, 3, 8, 0, 9, 1, 8, 1, 7, 4, 1, 6, 5, 6, 8, 9, 1, 3, 2, 7, 9, 2,
       7, 1, 5, 7, 2, 3, 1, 4, 2, 4, 0, 3, 3, 6, 6, 4, 1, 2, 3, 1, 3, 3,
       8, 1, 6, 8, 8, 6, 7, 5, 9, 5, 4, 9, 0, 0, 3, 8, 7, 9, 8, 8, 2, 7,
       3, 4, 6, 5, 5, 5, 7, 4, 3, 4, 1, 7, 4, 3, 4, 0, 5, 6, 0, 4, 0, 3,
       8, 7, 8, 4, 4, 6, 5, 0, 1, 1, 5, 2, 6, 8, 0, 5, 5, 0, 7, 5, 4, 3,
       1, 3, 9, 1, 0, 5, 7, 6, 9, 1, 4, 4, 8, 4, 2, 0, 1, 4, 0, 8, 6, 7,
       3, 4, 2, 1, 3, 8, 0, 0, 4, 7, 9, 8, 8, 2, 0, 5, 5, 2, 7, 2, 4, 3,
       1, 1, 5, 3, 8, 7, 0, 7, 0, 8, 5, 5, 0, 5, 5,